In [ ]:
from google.colab import files
uploaded=files.upload()

Saving HIGGS_train.csv to HIGGS_train.csv


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import KFold



#read data
df = pd.read_csv('HIGGS_train.csv', header=None)

#remove rows with nan or string values
df = df.apply(pd.to_numeric, errors='coerce')

# drop any rows with NaN values
df.dropna(inplace=True)

# convert the DataFrame to a NumPy array
dataset = df.to_numpy()
print(dataset)

#cast the values to float
X_train = X_train.astype(float)
Y_train = Y_train.astype(float)

#make sure there is no null values
print(np.isnan(X_train).any())
print(np.isnan(Y_train).any())

#Split data as input output
X = dataset[:, 1:]  
Y = dataset[:, 0] 
print (X.shape, Y.shape)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=42)
# X_train, X_cv, Y_train, Y_cv = train_test_split(X_train, Y_train, test_size=0.025, random_state=42)
print(type(X_train))

#normalize data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

#model architechture
model = Sequential([
  Dense(1200,input_dim=len(X_train[0,:]), activation='relu'),
  BatchNormalization(),
  Dropout(0.55),
  Dense(1200, activation='relu'),
  BatchNormalization(),
  Dropout(0.55),
  Dense(1200, activation='relu'),
  Dense(1, activation='sigmoid')
])

# define the callback function
early_stop = EarlyStopping(monitor='val_accuracy', patience=20)


In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'], jit_compile=True)
model.fit(X_train, Y_train, epochs=1000, batch_size=900, callbacks=[early_stop], validation_split=0.1)


Epoch 1/1000
540/540 [==============================] - 14s 13ms/step - loss: 0.6313 - accuracy: 0.6595 - val_loss: 0.5636 - val_accuracy: 0.7077
Epoch 2/1000
540/540 [==============================] - 3s 6ms/step - loss: 0.5648 - accuracy: 0.7054 - val_loss: 0.5433 - val_accuracy: 0.7227
Epoch 3/1000
540/540 [==============================] - 3s 6ms/step - loss: 0.5493 - accuracy: 0.7164 - val_loss: 0.5320 - val_accuracy: 0.7295
Epoch 4/1000
540/540 [==============================] - 3s 6ms/step - loss: 0.5396 - accuracy: 0.7233 - val_loss: 0.5215 - val_accuracy: 0.7359
Epoch 5/1000
540/540 [==============================] - 4s 7ms/step - loss: 0.5326 - accuracy: 0.7280 - val_loss: 0.5168 - val_accuracy: 0.7390
Epoch 6/1000
540/540 [==============================] - 3s 6ms/step - loss: 0.5269 - accuracy: 0.7328 - val_loss: 0.5129 - val_accuracy: 0.7406
Epoch 7/1000
540/540 [==============================] - 3s 6ms/step - loss: 0.5228 - accuracy: 0.7346 - val_loss: 0.5108 - val_accurac

In [ ]:
#save the model
import pickle
filename = 'model.pkl'
pickle.dump(model, open(filename, 'wb'))

#transform predictions to binary
def predict_binary(model, X):
    # Make predictions
    predictions = model.predict(X)
    # Apply threshold of 0.5
    binary_predictions = (predictions >= 0.51).astype(int)
    return binary_predictions

X_test = scaler.fit_transform(X_test)
binary_predictions = predict_binary(model, X_test)
accuracy = accuracy_score(Y_test,binary_predictions)
print(accuracy)

loaded_model = pickle.load(open(filename, 'rb'))


In [ ]:
from google.colab import files
files.download(filename)